In [1]:
# import all necessary references
import pandas as pd
import importlib.metadata
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import yaml
from pprint import pprint
from math import floor
from pathlib import PurePath, Path


print(

    f"""
Pandas version: {pd.__version__}
Matplotlib Pyplot version: {importlib.metadata.version("matplotlib")}
Seaborn version: {sns.__version__}
PyYAML version: {yaml.__version__}    
    """
)


Pandas version: 2.0.0
Matplotlib Pyplot version: 3.7.1
Seaborn version: 0.12.2
PyYAML version: 6.0    
    


In [2]:
# Pandas configuration
pd.set_option("display.max_rows", 150)
pd.set_option("display.max_columns", 150)
pd.set_option("display.max_colwidth", None)

# Plot configuration
%matplotlib widget

# Path configuration
current_directory = Path.cwd()
current_directory

PosixPath('/home/xlevb/projects/kaggle-home-credit-default-risk/notebooks')

In [3]:
def get_dir_path(dir_name: str):
    current_directory = Path.cwd()
    parents_dir = current_directory.parents

    for dir in parents_dir:
        for child_dir in dir.iterdir():
            if dir_name in child_dir.parts:
                return child_dir
    raise FileNotFoundError(f"Directory {dir_name!r} not found.")

In [4]:
# Create references for data paths
data_directory = get_dir_path("data")

raw_application_test_file = data_directory.joinpath("01_raw", "application_test.csv")
raw_application_train_file = data_directory.joinpath("01_raw", "application_train.csv")
raw_bureau_file = data_directory.joinpath("01_raw", "bureau.csv")
raw_bureau_balance_file = data_directory.joinpath("01_raw", "bureau_balance.csv")
raw_bureau_credit_card_balance_file = data_directory.joinpath("01_raw", "credit_card_balance.csv")
raw_installments_payments_file = data_directory.joinpath("01_raw", "credit_card_balance.csv")
raw_pos_cash_balance_file = data_directory.joinpath("01_raw", "POS_CASH_balance.csv")
raw_previous_application_file = data_directory.joinpath("01_raw", "previous_application.csv")
raw_sample_submission_file = data_directory.joinpath("01_raw", "sample_submission.csv")
home_credit_columns_description_file = data_directory.joinpath("01_raw", "HomeCredit_columns_description.csv")

conf_directory = get_dir_path("conf")
project_parameters_file = conf_directory.joinpath("base", "parameters.yml")

In [5]:
with open(project_parameters_file, "r") as file:
    project_parameters = yaml.safe_load(file)

pprint(project_parameters)

{'dataset_variables_types': {'application_test': {'categorical_nominal': ['FONDKAPREMONT_MODE',
                                                                          'FLAG_EMAIL',
                                                                          'WEEKDAY_APPR_PROCESS_START',
                                                                          'CODE_GENDER',
                                                                          'NAME_TYPE_SUITE',
                                                                          'FLAG_MOBIL',
                                                                          'NAME_EDUCATION_TYPE',
                                                                          'FLAG_PHONE',
                                                                          'EMERGENCYSTATE_MODE',
                                                                          'FLAG_OWN_REALTY',
                                                                   

In [6]:
home_credit_descriptors_df = pd.read_csv(
    filepath_or_buffer=home_credit_columns_description_file,
    engine="pyarrow",
    # dtype_backend="pyarrow",
    encoding='unicode_escape'
)
raw_application_train_df = pd.read_csv(
    filepath_or_buffer=raw_application_train_file,
    index_col="SK_ID_CURR",
    parse_dates=True,
    engine="pyarrow",
    # dtype_backend="pyarrow",
    encoding='unicode_escape'
)
raw_application_test_df = pd.read_csv(
    filepath_or_buffer=raw_application_test_file,
    index_col="SK_ID_CURR",
    parse_dates=True,
    engine="pyarrow",
    # dtype_backend="pyarrow",
    encoding='unicode_escape'
)

In [7]:
raw_application_train_df

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.0220,0.0198,0.0,0.0000,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.0000,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.0790,0.0554,0.0,0.0000,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.0100,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.7295

In [8]:
raw_application_train_df.shape

(307511, 121)

In [9]:
raw_application_train_df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
TARGET,307511.0,NaN,NaN,NaN,0.080729,0.272419,0.0,0.0,0.0,0.0,1.0
NAME_CONTRACT_TYPE,307511,2,Cash loans,278232,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CODE_GENDER,307511,3,F,202448,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FLAG_OWN_CAR,307511,2,N,202924,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FLAG_OWN_REALTY,307511,2,Y,213312,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CNT_CHILDREN,307511.0,NaN,NaN,NaN,0.417052,0.722121,0.0,0.0,0.0,1.0,19.0
AMT_INCOME_TOTAL,307511.0,NaN,NaN,NaN,168797.919297,237123.146279,25650.0,112500.0,147150.0,202500.0,117000000.0
AMT_CREDIT,307511.0,NaN,NaN,NaN,599025.999706,402490.776996,45000.0,270000.0,513531.0,808650.0,4050000.0
AMT_ANNUITY,307499.0,NaN,NaN,NaN,27108.573909,14493.737315,1615.5,16524.0,24903.0,34596.0,258025.5
AMT_GOODS_PRICE,307233.0,NaN,NaN,NaN,538396.207429,369446.46054,40500.0,238500.0,450000.0,679500.0,4050000.0


In [10]:
raw_application_train_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 307511 entries, 100002 to 456255
Data columns (total 121 columns):
 #    Column                        Dtype  
---   ------                        -----  
 0    TARGET                        int64  
 1    NAME_CONTRACT_TYPE            object 
 2    CODE_GENDER                   object 
 3    FLAG_OWN_CAR                  object 
 4    FLAG_OWN_REALTY               object 
 5    CNT_CHILDREN                  int64  
 6    AMT_INCOME_TOTAL              float64
 7    AMT_CREDIT                    float64
 8    AMT_ANNUITY                   float64
 9    AMT_GOODS_PRICE               float64
 10   NAME_TYPE_SUITE               object 
 11   NAME_INCOME_TYPE              object 
 12   NAME_EDUCATION_TYPE           object 
 13   NAME_FAMILY_STATUS            object 
 14   NAME_HOUSING_TYPE             object 
 15   REGION_POPULATION_RELATIVE    float64
 16   DAYS_BIRTH                    int64  
 17   DAYS_EMPLOYED                 int64  
 18   DA

In [11]:
# segregate descriptors ony for application dataset
application_descriptors = home_credit_descriptors_df[home_credit_descriptors_df["Table"]=="application_{train|test}.csv"]
application_descriptors.head(125)

,,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,
1,2,application_{train|test}.csv,TARGET,"Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)",
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,
5,8,application_{train|test}.csv,FLAG_OWN_REALTY,Flag if client owns a house or flat,
6,9,application_{train|test}.csv,CNT_CHILDREN,Number of children the client has,
7,10,application_{train|test}.csv,AMT_INCOME_TOTAL,Income of the client,
8,11,application_{train|test}.csv,AMT_CREDIT,Credit amount of the loan,
9,12,application_{train|test}.csv,AMT_ANNUITY,Loan annuity,


In [12]:
unique_id_variables = ["SK_ID_CURR"]

target_variables = ["TARGET"]

cat_variables = [
    col
    for col in raw_application_train_df.columns.to_list()
    if any(substring in col for substring in ["FLAG", "NAME", "RATING", "CODE"])
] + [
    "WEEKDAY_APPR_PROCESS_START",
    "FONDKAPREMONT_MODE",
    "HOUSETYPE_MODE",
    "NAME_TYPE_SUITE",
    "WALLSMATERIAL_MODE",
    "EMERGENCYSTATE_MODE",
]
cat_variables = list(
    set(cat_variables).difference(
        set(
            col
            for col in raw_application_train_df.columns.to_list()
            if any(substring in col for substring in ["FLAG_DOCUMENT"])
        )
    )
)

int_variables = [
    col
    for col in raw_application_train_df.columns.to_list()
    if any(substring in col for substring in ["AMT_REQ_CREDIT"])
]

float_variables = [
    col
    for col in raw_application_train_df.columns.to_list()
    if any(substring in col for substring in ["AMT_", "EXT_", "_AVG", "_MODE", "_MEDI"])
] + [
    "REGION_POPULATION_RELATIVE",
]
float_variables = list(
    set(float_variables).difference(set(cat_variables)).difference(set(int_variables))
)

time_interval_variables = [
    col
    for col in raw_application_train_df.columns.to_list()
    if any(
        substring in col for substring in ["YEARS", "MONTHS", "DAYS", "HOURS", "MINS"]
    )
]
time_interval_variables = list(
    set(time_interval_variables).difference(set(float_variables))
)

datetime_variables = [
    col
    for col in raw_application_train_df.columns.to_list()
    if any(substring in col for substring in ["YEAR", "MONTH", "DAY", "HOUR"])
]
datetime_variables = list(
    set(datetime_variables)
    .difference(set(cat_variables))
    .difference(set(int_variables))
    .difference(set(float_variables))
    .difference(set(time_interval_variables))
)


text_variables = []

print(
    f"""
Unique ID Variables: {unique_id_variables}

Target Variables: {target_variables}

Categorial Variables: {cat_variables}

Integer Variables: {int_variables}

Float Variables: {float_variables}

Time Interval Variables: {time_interval_variables}

Datetime Variables: {datetime_variables}

Text Variables: {text_variables}
"""
)


Unique ID Variables: ['SK_ID_CURR']

Target Variables: ['TARGET']

Categorial Variables: ['CODE_GENDER', 'FLAG_WORK_PHONE', 'FLAG_EMP_PHONE', 'NAME_CONTRACT_TYPE', 'FLAG_OWN_REALTY', 'WEEKDAY_APPR_PROCESS_START', 'FLAG_OWN_CAR', 'FLAG_MOBIL', 'HOUSETYPE_MODE', 'NAME_EDUCATION_TYPE', 'EMERGENCYSTATE_MODE', 'REGION_RATING_CLIENT_W_CITY', 'FONDKAPREMONT_MODE', 'FLAG_EMAIL', 'WALLSMATERIAL_MODE', 'FLAG_PHONE', 'NAME_INCOME_TYPE', 'REGION_RATING_CLIENT', 'NAME_HOUSING_TYPE', 'FLAG_CONT_MOBILE', 'NAME_FAMILY_STATUS', 'NAME_TYPE_SUITE']

Integer Variables: ['AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']

Float Variables: ['EXT_SOURCE_3', 'ENTRANCES_MODE', 'BASEMENTAREA_MODE', 'APARTMENTS_MEDI', 'YEARS_BUILD_MEDI', 'ENTRANCES_MEDI', 'NONLIVINGAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'ELEVATORS_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'AMT_INCOME_TOTAL', 'NONLIVING

In [13]:
cat_ordinal_vars = [
    "REGION_RATING_CLIENT",
    "REGION_RATING_CLIENT_W_CITY"
]

cat_nominal_vars = list(
    set(cat_variables)
    .difference(set(cat_ordinal_vars))
)

num_continuous_vars = float_variables.copy()

num_discrete_vars = int_variables + time_interval_variables



print(
    f"""
Categorial Ordinal Variables: {cat_ordinal_vars}

Categorial Nominal Variables: {cat_nominal_vars}

Numeric Continuous Variables: {num_continuous_vars}

Numeric Discrete Variables: {num_discrete_vars}

Datetime Variables: {datetime_variables}
"""
)


Categorial Ordinal Variables: ['REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY']

Categorial Nominal Variables: ['CODE_GENDER', 'FLAG_WORK_PHONE', 'FLAG_EMP_PHONE', 'NAME_CONTRACT_TYPE', 'FLAG_OWN_REALTY', 'WEEKDAY_APPR_PROCESS_START', 'FONDKAPREMONT_MODE', 'FLAG_EMAIL', 'WALLSMATERIAL_MODE', 'FLAG_OWN_CAR', 'FLAG_MOBIL', 'FLAG_PHONE', 'NAME_INCOME_TYPE', 'HOUSETYPE_MODE', 'NAME_HOUSING_TYPE', 'FLAG_CONT_MOBILE', 'NAME_EDUCATION_TYPE', 'EMERGENCYSTATE_MODE', 'NAME_FAMILY_STATUS', 'NAME_TYPE_SUITE']

Numeric Continuous Variables: ['EXT_SOURCE_3', 'ENTRANCES_MODE', 'BASEMENTAREA_MODE', 'APARTMENTS_MEDI', 'YEARS_BUILD_MEDI', 'ENTRANCES_MEDI', 'NONLIVINGAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'ELEVATORS_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'AMT_INCOME_TOTAL', 'NONLIVINGAPARTMENTS_MODE', 'ENTRANCES_AVG', 'FLOORSMIN_MODE', 'COMMONAREA_AVG', 'APARTMENTS_MODE', 'YEARS_BEGINEXPLUATATION_AVG', 'LANDAREA_MODE', 'FLOORSMAX_AVG', 'NONLIVINGAREA_MEDI', 'FLOORSMAX_MEDI', 'LIVINGAPARTMENTS

In [14]:
project_parameters["dataset_variables_types"] = {
    "application_train": {
        "categorical_ordinal": cat_ordinal_vars,
        "categorical_nominal": cat_nominal_vars,
        "numerical_continuous": num_continuous_vars,
        "numerical_discrete": num_discrete_vars,
        "datetime": datetime_variables
    },
    "application_test": {
        "categorical_ordinal": cat_ordinal_vars,
        "categorical_nominal": cat_nominal_vars,
        "numerical_continuous": num_continuous_vars,
        "numerical_discrete": num_discrete_vars,
        "datetime": datetime_variables
    }
}

project_parameters["target"] = target_variables

In [15]:
with open(project_parameters_file, "w") as file:
    yaml.safe_dump(project_parameters, file)

In [16]:
with open(project_parameters_file, "r") as file:
    project_parameters_updated = yaml.safe_load(file)

project_parameters_updated

{'dataset_variables_types': {'application_test': {'categorical_nominal': ['CODE_GENDER',
    'FLAG_WORK_PHONE',
    'FLAG_EMP_PHONE',
    'NAME_CONTRACT_TYPE',
    'FLAG_OWN_REALTY',
    'WEEKDAY_APPR_PROCESS_START',
    'FONDKAPREMONT_MODE',
    'FLAG_EMAIL',
    'WALLSMATERIAL_MODE',
    'FLAG_OWN_CAR',
    'FLAG_MOBIL',
    'FLAG_PHONE',
    'NAME_INCOME_TYPE',
    'HOUSETYPE_MODE',
    'NAME_HOUSING_TYPE',
    'FLAG_CONT_MOBILE',
    'NAME_EDUCATION_TYPE',
    'EMERGENCYSTATE_MODE',
    'NAME_FAMILY_STATUS',
    'NAME_TYPE_SUITE'],
   'categorical_ordinal': ['REGION_RATING_CLIENT',
    'REGION_RATING_CLIENT_W_CITY'],
   'datetime': ['HOUR_APPR_PROCESS_START'],
   'numerical_continuous': ['EXT_SOURCE_3',
    'ENTRANCES_MODE',
    'BASEMENTAREA_MODE',
    'APARTMENTS_MEDI',
    'YEARS_BUILD_MEDI',
    'ENTRANCES_MEDI',
    'NONLIVINGAREA_MODE',
    'YEARS_BEGINEXPLUATATION_MODE',
    'ELEVATORS_MEDI',
    'YEARS_BEGINEXPLUATATION_MEDI',
    'AMT_INCOME_TOTAL',
    'NONLIVINGAPARTMENT